# The Cat Expert Bot

Install the nesesary stuff

In [ ]:
# Installation command
%pip install -r requirements.txt


In [ ]:
#%pip install pytube
#%pip install --upgrade pytube
#%pip install yt-dlp
#%pip install moviepy
#%pip install whisper
#%pip install chromadb sentence-transformers
#%pip install git+https://github.com/openai/whisper.git
#%pip install pytubefix
#%pip install chromadb
#%pip install langchain
#%pip install openai
#%pip install opencv-python
#%pip install langchain_openai
#%pip install --upgrade huggingface_hub
#%pip install --upgrade sentence-transformers




# Load the key

In [3]:
from dotenv import load_dotenv
import os
import langsmith

# Specify the path to the .env file
dotenv_path = "./notebooks/apikey.env" #Change if your env is in a diffretn folder
load_dotenv(dotenv_path)

# Ensure required environment variables are loaded
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# Check if all environment variables are set; raise an error if any are missing
if not all([OPENAI_API_KEY, LANGCHAIN_API_KEY, HUGGINGFACEHUB_API_TOKEN]):
    raise ValueError("Some required API keys are missing in the .env file.")

# Enable LangSmith tracing with environment variables
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "cat_expert_knowledge"

# Initialize LangSmith Client
from langsmith import Client
client = Client(api_key=LANGCHAIN_API_KEY)


In [4]:
print (LANGCHAIN_API_KEY)
import os
os.path.abspath(os.getcwd())

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


'/notebooks'

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-555b8aef-49a0-46a4-9a95-fdb7163dd0de-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Load the videos to learn.
Video to Audio

In [6]:
from pytubefix import YouTube
import os
import hashlib

# Define the directory for downloaded audio files
output_path = "path/to/m4a"
os.makedirs(output_path, exist_ok=True)  # Ensure the directory exists

# Replace with your list of YouTube URLs
video_urls = ["https://youtu.be/ZUcVUFvmDFE?si=z9GfOAWF1qothiKs", 
            "https://youtu.be/4DlJYcfiRu4?si=cUVT9L5dEEdkcSt_", 
            "https://youtu.be/rxInrRQLEmM?si=Ai7wHN0dI--cns0x", 
            "https://youtu.be/gxlNfh5ukMw?si=naO3n4VZeXx3PlOs", 
            "https://youtu.be/ojS7XwtoXtw?si=NpNSef7dCm_LnFPv", 
            "https://youtu.be/tsYT7yIOdqQ?si=hdGEpxlmFNMf7NNQ", 
            "https://youtu.be/tsYT7yIOdqQ?si=e_Zdh2dGpqempHR8", 
            "https://youtu.be/UWohxDOXsl4?si=y1nXlUZYw6uzkc8n", 
            "https://youtu.be/gZrwcoiy_gY?si=ksfYE03t6xtuxUL0", 
            "https://youtu.be/lSDI5diNu4Y?si=Q-In6zMD4ZpuaPIz", 
            "https://youtu.be/8aCGL9GpVUg?si=_0yF1U1thjwJqyPY", 
            "https://youtu.be/VjOXvD7OvrE?si=t6xugNxLeMjpsi7E", 
            "https://youtu.be/FzifwTnCV5s?si=sR_u4kG-4NoQx5Ux", 
            "https://youtu.be/XreeFU7RYeI?si=hsc9WO24dJP6AfV2", 
            "https://youtu.be/-4O97jw_8Bc?si=pC14dgZ_f4mXdYPv" ]

# Function to create a unique filename from URL
def generate_filename(url, extension="m4a"):
    """Creates a unique filename for each URL based on its hash."""
    return hashlib.md5(url.encode()).hexdigest() + f".{extension}"

# Download each video as audio and handle errors
failed_downloads = []  # To log any failed downloads

for url in video_urls:
    # Generate filename and check if it exists
    filename = generate_filename(url)
    file_path = os.path.join(output_path, filename)
    
    if os.path.exists(file_path):
        print(f"Already downloaded: {url}")
        continue  # Skip downloading if file exists
    
    # Download video if not already downloaded
    try:
        yt = YouTube(url)
        video = yt.streams.filter(only_audio=True).first()
        video.download(output_path=output_path, filename=filename)
        print(f"Downloaded: {url}")
    except Exception as e:
        print(f"Failed to download {url}: {e}")
        failed_downloads.append(url)

# Optional: log failed downloads if any
if failed_downloads:
    print("Failed Downloads:", failed_downloads)
    # You could write these to a log file for later review

Already downloaded: https://youtu.be/ZUcVUFvmDFE?si=z9GfOAWF1qothiKs
Already downloaded: https://youtu.be/4DlJYcfiRu4?si=cUVT9L5dEEdkcSt_
Already downloaded: https://youtu.be/rxInrRQLEmM?si=Ai7wHN0dI--cns0x
Already downloaded: https://youtu.be/gxlNfh5ukMw?si=naO3n4VZeXx3PlOs
Already downloaded: https://youtu.be/ojS7XwtoXtw?si=NpNSef7dCm_LnFPv
Already downloaded: https://youtu.be/tsYT7yIOdqQ?si=hdGEpxlmFNMf7NNQ
Already downloaded: https://youtu.be/tsYT7yIOdqQ?si=e_Zdh2dGpqempHR8
Already downloaded: https://youtu.be/UWohxDOXsl4?si=y1nXlUZYw6uzkc8n
Already downloaded: https://youtu.be/gZrwcoiy_gY?si=ksfYE03t6xtuxUL0
Already downloaded: https://youtu.be/lSDI5diNu4Y?si=Q-In6zMD4ZpuaPIz
Already downloaded: https://youtu.be/8aCGL9GpVUg?si=_0yF1U1thjwJqyPY
Already downloaded: https://youtu.be/VjOXvD7OvrE?si=t6xugNxLeMjpsi7E
Already downloaded: https://youtu.be/FzifwTnCV5s?si=sR_u4kG-4NoQx5Ux
Already downloaded: https://youtu.be/XreeFU7RYeI?si=hsc9WO24dJP6AfV2
Already downloaded: https://youtu.

# Transcriptions done by Whisper

In [142]:
import whisper
import os

# Load Whisper model
model = whisper.load_model("base")

# Directory containing your .m4a files
audio_dir = "/notebooks/path/to/m4a/"
transcription_dir = "/notebooks/path/to/transcriptions/"

# Ensure the transcription directory exists
os.makedirs(transcription_dir, exist_ok=True)

# List all .m4a files in the audio directory
audio_files = [f for f in os.listdir(audio_dir) if f.endswith(".m4a")]

# Transcribe each audio file and save it as a .txt file
for audio_file in audio_files:
    audio_path = os.path.join(audio_dir, audio_file)
    try:
        transcription = model.transcribe(audio_path)

        # Save transcription to a .txt file
        
        transcription_file = os.path.join(transcription_dir, audio_file.replace(".m4a", ".txt"))
        with open(transcription_file, "w") as f:
            f.write(transcription['text'])
        print(f"Transcribed and saved: {transcription_file}")
    except Exception as e:
        print(f"Failed to transcribe {audio_file}: {e}")



Transcribed and saved: /notebooks/path/to/transcriptions/c4181c7e744aaaae5aaf7e234e39a17e.txt
Transcribed and saved: /notebooks/path/to/transcriptions/c813c650315977e7da699c0b2d52799d.txt
Transcribed and saved: /notebooks/path/to/transcriptions/When Cat Introductions Get UGLY.txt
Transcribed and saved: /notebooks/path/to/transcriptions/6e43a9a97ff958b094ee86b59d6fa433.txt
Transcribed and saved: /notebooks/path/to/transcriptions/Can my Cats Get Along? Cat-to-Cat Body Language basics & Introduction Tips.txt
Transcribed and saved: /notebooks/path/to/transcriptions/How To Introduce Your Cat to a New KITTEN!.txt
Transcribed and saved: /notebooks/path/to/transcriptions/How to Introduce Cats.txt
Transcribed and saved: /notebooks/path/to/transcriptions/Are My Cats Playing or Fighting? | Cat Playing vs Cat Aggression.txt
Transcribed and saved: /notebooks/path/to/transcriptions/Cat Introductions Gone Wrong: They Will NOT Work it Out Without You.txt
Transcribed and saved: /notebooks/path/to/trans

# Chromadb as Client and creation of Collection

In [7]:
import os
import glob

# Define the directory for transcription files
transcription_dir = "path/to/transcriptions"

# Check if the directory exists
if not os.path.exists(transcription_dir):
    print(f"The transcription directory {transcription_dir} does not exist.")
else:
    # List transcription files
    transcription_files = glob.glob(os.path.join(transcription_dir, "*.txt"))
    if not transcription_files:
        print(f"No .txt transcription files found in {transcription_dir}.")
    else:
        print(f"Found transcription files: {transcription_files}")


Found transcription files: ["path/to/transcriptions/The Do's & Don'ts of Introducing Cats.txt", 'path/to/transcriptions/c813c650315977e7da699c0b2d52799d.txt', 'path/to/transcriptions/55340c0bd9e5d16b88d7bdc38908eccc.txt', 'path/to/transcriptions/Can my Cats Get Along? Cat-to-Cat Body Language basics & Introduction Tips.txt', 'path/to/transcriptions/When Cat Introductions Get UGLY.txt', 'path/to/transcriptions/c4181c7e744aaaae5aaf7e234e39a17e.txt', 'path/to/transcriptions/fa8c74567ee31f6e4bdfc0abb681f0a4.txt', 'path/to/transcriptions/Brady Bunching: Introducing Two Groups of Cats.txt', 'path/to/transcriptions/Can Cats & Dogs Be Friends? | Jackson Galaxy.txt', 'path/to/transcriptions/b81b63d4601ef135d3641d91fa8e5920.txt', 'path/to/transcriptions/6e43a9a97ff958b094ee86b59d6fa433.txt', 'path/to/transcriptions/Cat Introductions: Good First Impressions are a must!.txt', 'path/to/transcriptions/How to Introduce Cats.txt', 'path/to/transcriptions/This was BAD!! 🙀🤯🫣 #shorts.txt', 'path/to/trans

In [8]:
import chromadb
from sentence_transformers import SentenceTransformer
import os
import glob
import hashlib

# Initialize ChromaDB client and set up the collection
client = chromadb.Client()

# Check if the collection exists; create if not
collection_name = "cat_expert_knowledge"
if collection_name not in [col.name for col in client.list_collections()]:
    collection = client.create_collection(collection_name)
else:
    collection = client.get_collection(collection_name)
    print(f"Using existing collection: {collection_name}")

# Load the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Directory containing transcription files
transcription_dir = "path/to/transcriptions"

def generate_document_id(text):
    """Generate a unique hash ID for each document based on its text content."""
    return hashlib.md5(text.encode()).hexdigest()

def add_document_to_chromadb(file_path, collection):
    """Adds a document to ChromaDB if it doesn't already exist."""
    # Read file content
    with open(file_path, "r") as f:
        text = f.read()
    
    # Generate a unique ID based on document content
    doc_id = generate_document_id(text)
    
    # Query to check if this document hash already exists
    existing_docs = collection.query(
        query_texts=[doc_id], 
        where={"topic": doc_id}
    )
    if existing_docs['documents']:  # If results found, skip adding
        print(f"Document with ID {doc_id} already exists in ChromaDB. Skipping.")
        return
    
    # Embed the text and add to ChromaDB
    embedding = embedding_model.encode(text).tolist()
    collection.add(
        ids=[doc_id],
        documents=[text],
        metadatas=[{"topic": doc_id, "source": file_path}],
        embeddings=[embedding]
    )
    print(f"Document {doc_id} added to ChromaDB.")

# Process each transcription file
transcription_files = glob.glob(os.path.join(transcription_dir, "*.txt"))
if not transcription_files:
    print("No transcription files found in the specified directory.")

for transcription_file in transcription_files:
    try:
        add_document_to_chromadb(transcription_file, collection)
    except Exception as e:
        print(f"Failed to add {transcription_file} to ChromaDB: {e}")


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 121MiB/s] 


Document with ID b64775dda1d19800faf705fc285d0b39 already exists in ChromaDB. Skipping.
Document with ID b64775dda1d19800faf705fc285d0b39 already exists in ChromaDB. Skipping.
Document with ID fd4d02a8b57e29119b6790372d447c5c already exists in ChromaDB. Skipping.
Document with ID f7a1458125d530bc7f93a9d9121cec6d already exists in ChromaDB. Skipping.
Document with ID 74bd2f092ea3b48dfd88b46de587a71f already exists in ChromaDB. Skipping.
Document with ID b472d1bcb7e3082fc1e88755987ab76a already exists in ChromaDB. Skipping.
Document with ID 099c99dd958dcdb0b42bf7a042932590 already exists in ChromaDB. Skipping.
Document with ID 68c751d8986d924ff7bbb9325eebec7c already exists in ChromaDB. Skipping.
Document with ID a015dbd299e90f7bf1f232372463a392 already exists in ChromaDB. Skipping.
Document with ID fd4d02a8b57e29119b6790372d447c5c already exists in ChromaDB. Skipping.
Document with ID 5535ae3ef6377118f115b87cf8582ab2 already exists in ChromaDB. Skipping.
Document with ID ab240479c60573c

Old code works 1

In [10]:
import chromadb
from sentence_transformers import SentenceTransformer
import glob

# Initialize ChromaDB client and create a collection
client = chromadb.Client()

# Check if the collection exists; create if not
if "cat_expert_knowledge" not in [col.name for col in client.list_collections()]:
    collection = client.create_collection("cat_expert_knowledge")
else:
    collection = client.get_collection("cat_expert_knowledge")


# Load the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load and process each transcription file
for idx, transcription_file in enumerate(glob.glob("/notebooks/path/to/transcriptions/*.txt")):
    with open(transcription_file, "r") as f:
        text = f.read()

    # Create an embedding for the transcription
    embedding = model.encode(text).tolist()
    topic = os.path.basename(transcription_file).replace(".txt", "")  # Use file name as topic

    # Add to ChromaDB collection with a unique ID
    collection.add(
        ids=[f"doc_{idx}"],  # Unique ID for each document
        documents=[text],
        metadatas=[{"topic": topic, "source": transcription_file}],
        embeddings=[embedding]
    )

print("Transcriptions successfully stored in ChromaDB!")

Transcriptions successfully stored in ChromaDB!


# Querying Chromadb
New code

In [11]:
from langchain_openai import ChatOpenAI

# Initialize the language model from LangChain
llm = ChatOpenAI()

def query_rag(collection, query_text, embedding_model, llm, n_results=3):
    """
    Perform a Retrieval-Augmented Generation (RAG) query to retrieve relevant documents
    and generate a response using a language model.
    
    Parameters:
    - collection: ChromaDB collection to query.
    - query_text: The question or input text for querying.
    - embedding_model: Sentence embedding model (e.g., SentenceTransformer).
    - llm: Language model from LangChain for generating responses.
    - n_results: Number of relevant documents to retrieve.
    
    Returns:
    - response: Generated answer from the language model based on retrieved documents.
    """
    
    # Embed the query text
    query_embedding = embedding_model.encode(query_text).tolist()
    
    # Perform the search in ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    
    # Check if documents are found
    if results['documents']:
        # Prepare the context from retrieved documents
        context_texts = [doc for doc in results['documents'][0]]
        context = "\n".join(context_texts)
        
        # Formulate prompt for the language model with context
        prompt = f"Context:\n{context}\n\nQuestion: {query_text}\nAnswer:"
        
        # Generate response using the language model
        response = llm.invoke(prompt)
        
        return response
    
    else:
        return "No relevant documents found for this query."

# Example Usage
query_text = "How do I stop my cat from scratching furniture?"
response = query_rag(collection, query_text, embedding_model, llm)
print("Generated Response:", response)


Generated Response: content="There are a few ways you can prevent your cat from scratching furniture. One option is to provide them with appropriate scratching posts or pads, so they can satisfy their scratching instinct in a designated area. You can also try using deterrents like double-sided tape or citrus spray on the furniture to discourage them from scratching. Additionally, you can trim your cat's nails regularly or use soft nail caps to prevent damage to furniture. Finally, positive reinforcement techniques, such as rewarding your cat for using the scratching post, can also be effective in redirecting their behavior." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 3555, 'total_tokens': 3665, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-

In [12]:
from sentence_transformers import SentenceTransformer

# Load the SentenceTransformer model for embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample query to retrieve relevant information
query_text = "How do I stop my cat from scratching furniture?"
query_embedding = embedding_model.encode(query_text).tolist()  # Correct embedding model

# Perform the search in ChromaDB
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3
)

# Check if results are found
if results['documents']:
    for result in results['documents']:
        print("Retrieved Text:", result)
    for metadata in results['metadatas']:
        print("Metadata:", metadata)
else:
    print("No relevant results found.")


Retrieved Text: [" What's up gang is I Jackson Galaxy your cat daddy and today your cat question answerer that is what I'm here for today hopefully we get questions all the time from folks just like you who sent it into this address right there which you of course can do as well and today we're gonna talk to Kinesha Kinesha what's it all about hi Jackson I'm Kinesha from the Cayman Islands my husband and I have a cat sunshine she's 11 years old and we've always wanted to get another cat to you know keep her company and also rescue from the human society but you know we're a bit worried because she has been the only cat in the home for so long in 11 years and we just wonder whether or not she'll be very receptive to the addition of another cat in the home we'd love your feedback on the best approach to do so you know what can we do to make this a seamless process because we'd really love for it to be a success and to avoid any elimination or scratching or you know any of the behaviors t

Old code but works 2

In [164]:
# Sample query to retrieve relevant information
query_text = "How do I stop my cat from scratching furniture?"
query_embedding = model.encode(query_text).tolist()

# Perform the search
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3
)

# Check if results are found
if results['documents']:
    for result in results['documents']:
        print("Retrieved Text:", result)
    for metadata in results['metadatas']:
        print("Metadata:", metadata)
else:
    print("No relevant results found.")



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved Text: [" What's up gang is I Jackson Galaxy your cat daddy and today your cat question answerer that is what I'm here for today hopefully we get questions all the time from folks just like you who sent it into this address right there which you of course can do as well and today we're gonna talk to Kinesha Kinesha what's it all about hi Jackson I'm Kinesha from the Cayman Islands my husband and I have a cat sunshine she's 11 years old and we've always wanted to get another cat to you know keep her company and also rescue from the human society but you know we're a bit worried because she has been the only cat in the home for so long in 11 years and we just wonder whether or not she'll be very receptive to the addition of another cat in the home we'd love your feedback on the best approach to do so you know what can we do to make this a seamless process because we'd really love for it to be a success and to avoid any elimination or scratching or you know any of the behaviors t

------------------------------

In [ ]:
%pip install langchain
%pip install openai
%pip install langchain_community langchain_openai

In [13]:
import langchain
langchain.__version__

'0.3.7'

In [19]:
# Initialize context history for conversation continuity
context_history = []

def retrieve_documents(query):
    # This example assumes you have a collection or database to query
    query_embedding = embedding_model.encode(query).tolist()
    result = collection.query(query_embeddings=[query_embedding], n_results=2)
    
    # Log retrieval details if using LangSmith
    langsmith.invoke("document_retrieval", {
        "query": query,
        "retrieved_documents": result['documents']
    })
    
    documents = " ".join(doc if isinstance(doc, str) else " ".join(doc) for doc in result['documents'])
    return documents


# LangSmith Check Point.
We define the functions of Langsmith to start tracing the project.

In [25]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence

def create_qa_chain():
    template = """You are a friendly, expert cat behaviorist inspired by Jackson Galaxy.
                  Use the information below to provide advice in a helpful and engaging way."""
    prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("user", "{context}\n\nQuestion: {question}\nAnswer:")
    ])
    
    # Log QA chain creation
    langsmith.invoke("qa_chain_creation", {
        "template": template,
    })
    
    return RunnableSequence(prompt | llm)


In [26]:
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.stdout import StdOutCallbackHandler
import langsmith


# Initialize Tracer callback handler for LangChain's built-in tracing
tracer_handler = StdOutCallbackHandler()

# Attach the callback handler to a CallbackManager
callback_manager = CallbackManager([tracer_handler])

# Main agent function with LangChain tracing
def ask_agent(query):
    # Append the new question to context history
    context_history.append(f"Question: {query}")

    # Retrieve relevant context from ChromaDB based on query variations
    conversation_context = retrieve_documents(query)
    
    # Combine recent exchanges for conversational memory
    recent_history = "\n".join(context_history[-3:])  # Keep last 3 exchanges for continuity
    combined_context = recent_history + "\n\n" + conversation_context
    
    # Create QA chain with the combined context
    qa_chain = create_qa_chain()
    answer = qa_chain.run({
        "context": combined_context,
        "question": query
    })

    # Log combined context for QA
    langsmith.invoke("combined_context_for_qa", {
        "recent_history": recent_history,
        "conversation_context": conversation_context,
        "combined_context": combined_context
    })

    # Use callback_manager to log the QA chain run
    answer = qa_chain.run(
        {"context": combined_context, "question": query},
        callbacks=callback_manager
    )

 # Log final answer
    langsmith.invoke("final_answer", {
        "query": query,
        "combined_context": combined_context,
        "answer": answer
    })
    # Append the answer to the context history for continuity
    context_history.append(f"Answer: {answer}")
    return answer
   


# Model
Doing A query in character.

In [22]:
%pip install langchain_community


In [30]:
import logging
from langchain.prompts import ChatPromptTemplate
from sentence_transformers import SentenceTransformer
import chromadb
import os
from dotenv import load_dotenv
from langchain_core.runnables import RunnableSequence

# Initialize model and embeddings
llm = ChatOpenAI(model="gpt-3.5-turbo")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
client = chromadb.Client()
collection = client.get_collection("cat_expert_knowledge")

# Setup logging
logging.basicConfig(level=logging.INFO, filename="agent_logs.log", filemode="a",
                    format="%(asctime)s - %(levelname)s - %(message)s")

# Helper function for document retrieval
def retrieve_documents(query):
    query_embedding = embedding_model.encode(query).tolist()
    result = collection.query(query_embeddings=[query_embedding], n_results=2)
    documents = " ".join(doc if isinstance(doc, str) else " ".join(doc) for doc in result['documents'])
    return documents

# QA Chain Creation
def create_qa_chain():
    template = """You are a friendly, expert cat behaviorist inspired by Jackson Galaxy.
                  Use the information below to provide advice in a helpful and engaging way."""
    prompt = ChatPromptTemplate.from_messages([("system", template), ("user", "{context}\n\nQuestion: {question}\nAnswer:")])
    return RunnableSequence(prompt | llm)

# Agent function with manual logging
def ask_agent(query):
    # Log the incoming query
    logging.info(f"User Query: {query}")

    # Retrieve context and answer
    conversation_context = retrieve_documents(query)
    qa_chain = create_qa_chain()
    answer = qa_chain.invoke({"context": conversation_context, "question": query})

    # Log the generated answer
    logging.info(f"Agent Response: {answer}")
    return answer

# Test with a sample query
user_query = "Why does my cat scratch furniture?"
response = ask_agent(user_query)
print("Agent Response:", response)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Agent Response: content="Hey there, Kinesha! It's great to hear that you're considering bringing another cat into your home to keep Sunshine company. It's wonderful that you're thinking of adopting from a shelter - that's a fantastic way to give a cat a second chance at a happy life.\n\nNow, let's talk about why cats scratch furniture. Scratching is a natural behavior for cats. It helps them stretch their muscles, sharpen their claws, and mark their territory with scent glands in their paws. So, when Sunshine scratches your furniture, she's not trying to be naughty - she's just following her instincts.\n\nTo prevent furniture scratching, you can provide alternative scratching surfaces for Sunshine. Invest in a sturdy scratching post made of materials like sisal or cardboard. Place the scratching post near her favorite furniture so she's more likely to use it.\n\nAnother tip is to make the furniture less appealing to scratch. You can do this by covering it with double-sided tape or usin

Langsmith is allready getting results in character ^

In [29]:
from langchain_openai import ChatOpenAI

# Initialize the language model from LangChain
llm = ChatOpenAI()

def query_rag(collection, query_text, model, llm, n_results=3):
    """
    Perform a Retrieval-Augmented Generation (RAG) query to retrieve relevant documents
    and generate a response using a language model.
    
    Parameters:
    - collection: ChromaDB collection to query.
    - query_text: The question or input text for querying.
    - model: Sentence embedding model (e.g., SentenceTransformer).
    - llm: Language model from LangChain for generating responses.
    - n_results: Number of relevant documents to retrieve.
    
    Returns:
    - response: Generated answer from the language model based on retrieved documents.
    """
    
    # Embed the query text
    query_embedding = model.encode(query_text).tolist()
    
    # Perform the search in ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    
    # Check if documents are found
    if results['documents']:
        # Prepare the context from retrieved documents
        context_texts = [doc for doc in results['documents'][0]]
        context = "\n".join(context_texts)
        
        # Formulate prompt for the language model with context
        prompt = f"Context:\n{context}\n\nQuestion: {query_text}\nAnswer:"
        
        # Generate response using the language model
        response = llm.invoke(prompt)
        
        return response
    
    else:
        return "No relevant documents found for this query."

# Example Usage
query_text = "How do I introduce my cat to another cat?"
response = query_rag(collection, query_text, model, llm)
print("Generated Response:", response)

# Example Usage
query_text = "How do I stop my cat from scratching furniture?"
response = query_rag(collection, query_text, model, llm)
print("Generated Response:", response)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated Response: content='To introduce your cat to another cat, it\'s important to follow a gradual and structured process. Start by setting up a base camp for the new cat, where they can have their own space with familiar scents and items. Make sure your existing cat is on a meal schedule and incorporate playtime into their routine. \n\nNext, begin sight swapping, where the cats can get used to each other\'s scents without direct contact. Gradually increase visual access by using a screen door or baby gates to create a safe barrier. \n\nImplement a feeding ritual where the cats can associate each other\'s presence with positive experiences like mealtime. Gradually reduce the distance between the cats during feeding sessions to build comfort and familiarity. \n\nFinally, engage in the "eat play love" exercise where both cats can interact in a positive and playful manner. Monitor their interactions and be prepared to separate them if needed. \n\nOverall, the key is to take things slo

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated Response: content="There are several ways to stop your cat from scratching furniture. One option is to provide your cat with scratching posts or pads made of materials like sisal or cardboard, which can satisfy their natural scratching instincts. You can also try using deterrents like double-sided tape or citrus scents on the furniture to discourage scratching. Additionally, trimming your cat's nails regularly and providing them with plenty of toys and enrichment can help redirect their scratching behavior. It's important to be patient and consistent in training your cat to use appropriate scratching surfaces." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 3555, 'total_tokens': 3660, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-

# Store Embeddings and Perform Retrieval:
Since LangSmith doesn’t yet support a full embedding storage and retrieval service like ChromaDB, we’ll use numpy to store and retrieve embeddings locally. This will simulate a vector search.

In [31]:
from sentence_transformers import SentenceTransformer

# Initialize the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Example text data
text_data = [
    {"id": "doc1", "content": "Why do cats scratch furniture?"},
    {"id": "doc2", "content": "How to introduce a new cat to your home."},
    # Add more text entries as needed
]

# Generate embeddings for each text entry
text_embeddings = [
    {"id": entry["id"], "embedding": embedding_model.encode(entry["content"]).tolist()}
    for entry in text_data
]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [32]:
# Store only text embeddings in a dictionary for quick access
text_store = {entry["id"]: entry for entry in text_embeddings}


In [33]:
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_similar_embeddings(query, store, top_k=2):
    """Retrieve top K similar text embeddings from the store."""
    query_embedding = clip_embd.embed_documents([query])
    similarities = []

    for item in store.values():
        score = cosine_similarity(query_embedding, item["embedding"]).flatten()[0]
        similarities.append((score, item))

    # Sort by similarity and select top K results
    similarities = sorted(similarities, key=lambda x: x[0], reverse=True)
    return [item for _, item in similarities[:top_k]]


In [ ]:
from sentence_transformers import SentenceTransformer

# Initialize the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Example text data
text_data = [
    {"id": "doc1", "content": "Why do cats scratch furniture?"},
    {"id": "doc2", "content": "How to introduce a new cat to your home."},
    # Add more text entries as needed
]

# Generate embeddings for each text entry
text_embeddings = [
    {"id": entry["id"], "embedding": embedding_model.encode(entry["content"]).tolist()}
    for entry in text_data
]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

# Update LangChain Agent for Text Retrieval

Modify the agent to retrieve and present both text and images, enhancing responses when visual examples are available.

Update the Retrieval Function to Include Images:

Adjust retrieve_documents to search for both text and image embeddings.

In [34]:
def ask_agent(query):
    # Retrieve similar text embeddings
    relevant_texts = retrieve_similar_embeddings(query, text_store)
    
    # Combine text responses
    text_response = " ".join([text["metadata"]["text"] for text in relevant_texts])
    
    # Return text-only response
    response = f"Expert Advice:\n{text_response}"
    return response


# Optimize Multimodal Retrieval and Response Formatting

In [35]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def retrieve_similar_embeddings(query, store, top_k=2):
    """Retrieve top K similar text embeddings from the store."""
    # Generate the embedding for the query using SentenceTransformer
    query_embedding = embedding_model.encode([query])
    
    similarities = []
    for item in store.values():
        # Compute cosine similarity between query embedding and each stored embedding
        score = cosine_similarity(query_embedding, [item["embedding"]]).flatten()[0]
        similarities.append((score, item))
    
    # Sort by similarity score in descending order and select top K results
    similarities = sorted(similarities, key=lambda x: x[0], reverse=True)
    return [item for _, item in similarities[:top_k]]


In [36]:
from sentence_transformers import SentenceTransformer

# Initialize the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Example text data
text_data = [
    {"id": "doc1", "content": "Why do cats scratch furniture?"},
    {"id": "doc2", "content": "How to introduce a new cat to your home."},
    # Add more text entries as needed
]

# Generate embeddings for each text entry
text_embeddings = [
    {"id": entry["id"], "embedding": embedding_model.encode(entry["content"]).tolist()}
    for entry in text_data
]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [37]:
def ask_agent(query):
    # Retrieve similar text embeddings
    relevant_texts = retrieve_similar_embeddings(query, text_store)
    
    # Combine text responses
    text_response = " ".join([text["content"] for text in relevant_texts])
    
    # Final response with formatted sections
    response = (
        f"**Expert Advice:**\n{text_response}\n\n"
    )
    return response

# Test the agent with a sample query
user_query = "How can I create a cat-friendly environment?"
response = ask_agent(user_query)
print("Agent Response:", response)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

KeyError: 'content'

# Implement a Feedback Loop

In [ ]:
import json

# Define a function to log feedback
feedback_log = "feedback_log.json"  # Path to store feedback

def log_feedback(query, response, rating, comment=None):
    feedback_entry = {
        "query": query,
        "response": response,
        "rating": rating,
        "comment": comment
    }
    
    # Append feedback to a JSON file
    try:
        with open(feedback_log, "a") as f:
            f.write(json.dumps(feedback_entry) + "\n")
    except IOError:
        print("Error logging feedback.")

def collect_feedback(query, response):
    print("Rate the helpfulness of the response (1-5):")
    rating = int(input("Rating: "))
    comment = input("Any comments? ")

# Log the feedback
    langsmith.invoke("user_feedback", {
        "query": query,
        "response": response,
        "rating": rating,
        "comment": comment
    })

# Append feedback to JSON log
    log_feedback(query, response, rating, comment)
    print("Thank you for your feedback!")

# Test the agent with feedback
#user_query = "Why does my cat scratch furniture?"
#response = ask_agent(user_query)
#print("Agent Response:", response)

# Collect feedback from user
#collect_feedback(user_query, response)



# Implement Personalization

In [ ]:
# Dictionary to store user profiles
user_profiles = {}

# Function to initialize or update a user profile
def get_user_profile(user_id):
    if user_id not in user_profiles:
        user_profiles[user_id] = {
            "name": "User",
            "preferred_response_length": "detailed",  # or 'concise'
            "interest_areas": ["behavior", "environment"]
        }
    return user_profiles[user_id]

# Function to update profile details
def update_user_profile(user_id, name=None, response_length=None, interests=None):
    profile = get_user_profile(user_id)
    if name:
        profile["name"] = name
    if response_length:
        profile["preferred_response_length"] = response_length
    if interests:
        profile["interest_areas"] = interests


# Optimize Memory Management

In [ ]:
# Limit memory length to relevant recent exchanges
def manage_memory(query, user_id, max_exchanges=3):
    profile = get_user_profile(user_id)
    relevant_context = "\n".join([context for context in context_history[-max_exchanges:] 
                                  if any(area in context for area in profile["interest_areas"])])
    return relevant_context


# Intent Handling Detection

We’ll add a simple intent classification to help the chatbot recognize and respond to different types of user intents. Basic intents might include requests for:

Advice: General guidance or recommendations.
Visual Examples: Requests for images or visual aids.
Detailed Explanations: In-depth responses for complex questions.
Define Intent Detection
For simplicity, we’ll use keywords to detect intents. For more robust intent classification, you could use a trained NLP model.

In [ ]:
def detect_intent(query):
    intent = "advice"  # Default intent
    if any(word in query for word in ["how", "why"]):
        intent = "advice"
    elif any(word in query for word in ["show", "picture", "visual", "image"]):
        intent = "visual_example"
    elif any(word in query for word in ["explain", "detailed", "in-depth"]):
        intent = "detailed_explanation"
    
    # LangSmith invoke to log intent detection
    langsmith.invoke("intent_detection", {"query": query, "detected_intent": intent})
    
    return intent


# Modify ask_agent to Use Personalization, Memory Management, and Intent Handling
Update the main function to:

Retrieve the user’s profile.
Use the detected intent to adjust the response.
Apply memory management to provide relevant context.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Mock storage for embeddings (use actual embeddings in a real setup)
text_store = {}
image_store = {}

# Define `retrieve_documents` function
def retrieve_documents(query, top_k=2):
    query_embedding = clip_embd.embed_documents([query])
    text_results, image_results = [], []
    
    # Retrieve text and image embeddings
    for item in text_store.values():
        score = cosine_similarity(query_embedding, item["embedding"]).flatten()[0]
        text_results.append((score, item))
    text_results = sorted(text_results, key=lambda x: x[0], reverse=True)[:top_k]
    
   
    relevant_texts = [item[1] for item in text_results]
    
    # LangSmith invoke to log retrieval details
    langsmith.invoke("retrieval", {
        "query": query,
        "text_results": [t[1]["text"] for t in text_results],
    })
    
    return relevant_texts, relevant_images


In [ ]:
%pip install --upgrade gradio
%pip install SpeechRecognition
%pip install gTTS




# Gradio deployment

In [ ]:
import gradio as gr
import speech_recognition as sr
from gtts import gTTS
import tempfile
from main import ask_agent  # Import the ask_agent function

# Convert audio to text using Google Speech Recognition
def audio_to_text(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            return text
        except sr.UnknownValueError:
            return "Sorry, I could not understand the audio."
        except sr.RequestError:
            return "Could not request results; check your internet connection."

# Convert text to speech using gTTS
def handle_input(input_text=None, audio_file=None):
    transcribed_text = input_text
    if audio_file is not None:
        transcribed_text = audio_to_text(audio_file)
        if transcribed_text.startswith("Sorry"):
            # Log failed transcription
            langsmith.invoke("transcription_error", {
                "input_type": "audio",
                "transcribed_text": transcribed_text
            })
            return transcribed_text, None

    # Log user's text input
    langsmith.invoke("user_input", {
        "input_type": "text" if audio_file is None else "audio",
        "text": transcribed_text
    })

    # Call ask_agent and log the response
    response_text = ask_agent(transcribed_text)
    langsmith.invoke("final_response", {
        "user_query": transcribed_text,
        "response": response_text
    })

    audio_response = text_to_speech_gtts(response_text)
    return response_text, audio_response




# Create Gradio interface
iface = gr.Interface(
    fn=handle_input,
    inputs=[
        gr.Textbox(placeholder="Type your question here...", label="Text Input"),
        gr.Audio(type="filepath", label="Or speak your question")
    ],
    outputs=[
        gr.Textbox(label="Cat Expert's Response"),
        gr.Audio(label="Spoken Response")
    ],
    title="Cat Behavior Expert Chatbot",
    description="Ask questions about cat behavior, either by typing or speaking.",
)

# Launch the interface
iface.launch(share=True)


ModuleNotFoundError: No module named 'main'